In [1]:
import os

In [3]:
%pwd

'/workspaces/End-to-End-MLOps-Bootcamp-Build-Deploy-and-Automate-ML-with-Data-Science-Projects/myFirstNewProject/research'

In [4]:
# changing it the to the root directory
os.chdir("../")

In [5]:
%pwd

'/workspaces/End-to-End-MLOps-Bootcamp-Build-Deploy-and-Automate-ML-with-Data-Science-Projects/myFirstNewProject'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_data_file: Path
   unzip_dir: Path

In [ ]:
from src.my_first_end_to_end_project.constants import *
from src.my_first_end_to_end_project.utils.common_utils import read_yaml, create_directories


class ConfigurationManager:

    # reading the yaml files and creating necessary files
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath =  PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                     ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    # creating the Pipeline
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

    

In [ ]:
## New component class
import urllib.request as request
from src.my_first_end_to_end_project.logger import logger 
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config= config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info : {headers}")
        else:
            logger.info(f"The file already exists in the local repository")
